# Data Preparation Pipeline - Preparación Final para ML
**Sistema de Anotación de Video - Entrega 2**

Este notebook implementa la preparación completa de datos para entrenamiento de modelos ML **sin data leakage**.

## Proceso Completo:
1. **Cargar 3 datasets separados** (train/val/test del Notebook 3)
2. **Feature Engineering** (crear características geométricas)
3. **Pipeline de preprocessing** (normalización SOLO en train)
4. **Validación y guardar** archivos finales para modelado

## Input:
- `train_balanced.csv` (5,418 frames con SMOTE)
- `val_original.csv` (967 frames 100% reales)
- `test_original.csv` (967 frames 100% reales)

## Output:
- **X_train, y_train** (preprocesados con PCA)
- **X_val, y_val** (transformados con pipeline de train)
- **X_test, y_test** (transformados con pipeline de train)
- **Pipelines guardados** (scaler, label_encoder, pca)

## ⚠️ GARANTÍA:

✅ **Sin data leakage:** Val/Test nunca participan en fit()

✅ **Balance correcto:** Train 0.800, Val/Test 0.509

✅ **Reproducible:** Pipelines guardados para producción

---


## Paso 1: Instalación y Setup
Instalar librerías necesarias para feature engineering y preprocessing.


In [1]:
# ============================================
# PASO 1: INSTALACIÓN Y SETUP
# ============================================

# Instalar dependencias
!pip install pandas numpy matplotlib seaborn scikit-learn joblib -q

# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score, StratifiedKFold
import joblib
import json
import os
import sklearn
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías para data preparation cargadas")
print(f"📊 Pandas: {pd.__version__}")
print(f"🔬 Scikit-learn: {sklearn.__version__}")
print(f"💾 Joblib: {joblib.__version__}")


✅ Librerías para data preparation cargadas
📊 Pandas: 2.2.2
🔬 Scikit-learn: 1.6.1
💾 Joblib: 1.5.2


## Paso 2: Cargar Dataset Aumentado
Cargar el dataset balanceado y aumentado desde el Notebook 3.


In [3]:
# ============================================
# PASO 2: CARGAR DATASETS SEPARADOS (CORREGIDO)
# ============================================

def load_prepared_datasets():
    """
    Cargar los 3 datasets del Notebook 3 (YA SEPARADOS)
    """
    print("\n📂 CARGANDO DATASETS PREPARADOS (NOTEBOOK 3)")
    print("=" * 60)

    # Rutas de archivos
    train_path = 'train_balanced.csv'
    val_path = 'val_original.csv'
    test_path = 'test_original.csv'

    # Verificar existencia
    missing_files = []
    for path in [train_path, val_path, test_path]:
        if not os.path.exists(path):
            missing_files.append(path)

    if missing_files:
        print(f"❌ ARCHIVOS NO ENCONTRADOS:")
        for f in missing_files:
            print(f"   • {f}")
        return None, None, None

    # Cargar CSVs
    print(f"📁 Cargando archivos...")
    train_df = pd.read_csv(train_path)
    val_df = pd.read_csv(val_path)
    test_df = pd.read_csv(test_path)

    print(f"\n✅ DATASETS CARGADOS:")
    print(f"   Train: {len(train_df):,} frames")
    print(f"   Validation: {len(val_df):,} frames")
    print(f"   Test: {len(test_df):,} frames")
    print(f"   Total: {len(train_df) + len(val_df) + len(test_df):,} frames")

    # Verificar estructura
    print(f"\n📋 VERIFICACIÓN DE ESTRUCTURA:")
    print(f"   Columnas train: {len(train_df.columns)}")
    print(f"   Columnas val: {len(val_df.columns)}")
    print(f"   Columnas test: {len(test_df.columns)}")

    # Verificar composición de TRAIN
    if 'data_type' in train_df.columns:
        print(f"\n🔍 COMPOSICIÓN TRAIN:")
        for dtype, count in train_df['data_type'].value_counts().items():
            pct = count / len(train_df) * 100
            print(f"   {dtype:10s}: {count:5,} ({pct:4.1f}%)")

    # Verificar balance
    print(f"\n⚖️  BALANCE POR DATASET:")
    for name, df in [('Train', train_df), ('Val', val_df), ('Test', test_df)]:
        counts = df['activity'].value_counts()
        balance = counts.min() / counts.max()

        print(f"\n   {name}:")
        for activity, count in counts.items():
            pct = count / len(df) * 100
            print(f"      {activity:20s}: {count:4,} ({pct:4.1f}%)")
        print(f"      Balance ratio: {balance:.3f}")

    # Verificar data leakage (SOLO si existe video_file)
    print(f"\n🔒 VERIFICACIÓN DE LEAKAGE:")

    # Chequear qué datasets tienen video_file
    has_video_file = {
        'train': 'video_file' in train_df.columns,
        'val': 'video_file' in val_df.columns,
        'test': 'video_file' in test_df.columns
    }

    if all(has_video_file.values()):
        # Todos tienen video_file, verificar overlap
        train_videos = set(train_df['video_file'].unique())
        val_videos = set(val_df['video_file'].unique())
        test_videos = set(test_df['video_file'].unique())

        overlap_train_val = train_videos & val_videos
        overlap_train_test = train_videos & test_videos
        overlap_val_test = val_videos & test_videos

        if overlap_train_val or overlap_train_test or overlap_val_test:
            print(f"   ⚠️  POSIBLE LEAKAGE DETECTADO:")
            if overlap_train_val:
                print(f"      Train-Val overlap: {len(overlap_train_val)} videos")
            if overlap_train_test:
                print(f"      Train-Test overlap: {len(overlap_train_test)} videos")
            if overlap_val_test:
                print(f"      Val-Test overlap: {len(overlap_val_test)} videos")
        else:
            print(f"   ✅ SIN LEAKAGE: Videos únicos en cada split")
    else:
        # Algunos no tienen video_file
        print(f"   ℹ️  Columna 'video_file' no disponible en todos los datasets:")
        for name, has_col in has_video_file.items():
            status = "✅" if has_col else "❌"
            print(f"      {status} {name}")
        print(f"   💡 Verificación de leakage omitida")
        print(f"   ✅ Datasets ya están correctamente separados por Notebook 3")

    print(f"\n✅ DATASETS LISTOS PARA FEATURE ENGINEERING")

    return train_df, val_df, test_df

# EJECUTAR CARGA
train_df, val_df, test_df = load_prepared_datasets()

if all(df is not None for df in [train_df, val_df, test_df]):
    print(f"\n🎉 CARGA EXITOSA")

    # Mostrar columnas únicas para debug
    print(f"\n🔍 DIFERENCIAS DE COLUMNAS:")
    train_cols = set(train_df.columns)
    val_cols = set(val_df.columns)
    test_cols = set(test_df.columns)

    only_in_train = train_cols - val_cols - test_cols
    if only_in_train:
        print(f"   📋 Solo en train: {only_in_train}")

    common_cols = train_cols & val_cols & test_cols
    print(f"   ✅ Columnas comunes: {len(common_cols)}")

    print(f"\n🚀 Listo para continuar con Feature Engineering")
else:
    print(f"\n❌ ERROR EN CARGA")



📂 CARGANDO DATASETS PREPARADOS (NOTEBOOK 3)
📁 Cargando archivos...

✅ DATASETS CARGADOS:
   Train: 5,418 frames
   Validation: 967 frames
   Test: 967 frames
   Total: 7,352 frames

📋 VERIFICACIÓN DE ESTRUCTURA:
   Columnas train: 67
   Columnas val: 66
   Columnas test: 66

🔍 COMPOSICIÓN TRAIN:
   original  : 4,509 (83.2%)
   smote     :   909 (16.8%)

⚖️  BALANCE POR DATASET:

   Train:
      caminar_hacia       : 1,290 (23.8%)
      girar               : 1,032 (19.0%)
      ponerse_pie         : 1,032 (19.0%)
      sentarse            : 1,032 (19.0%)
      caminar_regreso     : 1,032 (19.0%)
      Balance ratio: 0.800

   Val:
      caminar_hacia       :  277 (28.6%)
      caminar_regreso     :  195 (20.2%)
      sentarse            :  188 (19.4%)
      ponerse_pie         :  166 (17.2%)
      girar               :  141 (14.6%)
      Balance ratio: 0.509

   Test:
      caminar_hacia       :  277 (28.6%)
      caminar_regreso     :  195 (20.2%)
      sentarse            :  188 (19.

## Paso 3: Feature Engineering - Características Geométricas
Crear características geométricas avanzadas a partir de landmarks existentes.


In [4]:
# ============================================
# PASO 3: FEATURE ENGINEERING (SIN LEAKAGE)
# ============================================

class GeometricFeatureEngineer:
    """Crear características geométricas desde landmarks"""

    def __init__(self):
        # Definir landmarks principales
        self.landmarks = {
            'L_shoulder': ['L_shoulder_x', 'L_shoulder_y', 'L_shoulder_z'],
            'R_shoulder': ['R_shoulder_x', 'R_shoulder_y', 'R_shoulder_z'],
            'L_elbow': ['L_elbow_x', 'L_elbow_y', 'L_elbow_z'],
            'R_elbow': ['R_elbow_x', 'R_elbow_y', 'R_elbow_z'],
            'L_wrist': ['L_wrist_x', 'L_wrist_y', 'L_wrist_z'],
            'R_wrist': ['R_wrist_x', 'R_wrist_y', 'R_wrist_z'],
            'L_hip': ['L_hip_x', 'L_hip_y', 'L_hip_z'],
            'R_hip': ['R_hip_x', 'R_hip_y', 'R_hip_z'],
            'L_knee': ['L_knee_x', 'L_knee_y', 'L_knee_z'],
            'R_knee': ['R_knee_x', 'R_knee_y', 'R_knee_z'],
            'L_ankle': ['L_ankle_x', 'L_ankle_y', 'L_ankle_z'],
            'R_ankle': ['R_ankle_x', 'R_ankle_y', 'R_ankle_z']
        }

    def create_geometric_features(self, df, dataset_name='Dataset'):
        """Crear todas las características geométricas"""
        print(f"\n🔶 FEATURE ENGINEERING: {dataset_name}")
        print("=" * 60)

        df_geo = df.copy()
        features_created = []

        # 1. Distancias entre landmarks
        distances = self._create_distance_features(df_geo)
        features_created.extend(distances)

        # 2. Ángulos articulares
        angles = self._create_angle_features(df_geo)
        features_created.extend(angles)

        # 3. Ratios corporales
        ratios = self._create_ratio_features(df_geo)
        features_created.extend(ratios)

        # 4. Centro de masa
        center_features = self._create_center_mass_features(df_geo)
        features_created.extend(center_features)

        print(f"\n✅ FEATURES CREADAS PARA {dataset_name}:")
        print(f"   📊 Total nuevas features: {len(features_created)}")
        print(f"   📐 Dimensiones finales: {df_geo.shape[0]:,} × {df_geo.shape[1]}")

        return df_geo, features_created

    def _create_distance_features(self, df):
        """Crear características de distancias"""
        print("   📏 Creando distancias entre landmarks...")

        distances_created = []

        # Distancias corporales importantes
        distance_pairs = [
            ('L_shoulder', 'R_shoulder', 'shoulder_width'),
            ('L_hip', 'R_hip', 'hip_width'),
            ('L_shoulder', 'L_hip', 'L_torso_length'),
            ('R_shoulder', 'R_hip', 'R_torso_length'),
            ('L_hip', 'L_knee', 'L_thigh_length'),
            ('R_hip', 'R_knee', 'R_thigh_length'),
            ('L_knee', 'L_ankle', 'L_shin_length'),
            ('R_knee', 'R_ankle', 'R_shin_length')
        ]

        for lm1, lm2, feature_name in distance_pairs:
            if all(col in df.columns for col in self.landmarks[lm1]) and \
               all(col in df.columns for col in self.landmarks[lm2]):

                # Calcular distancia euclidiana 3D
                x1, y1, z1 = df[self.landmarks[lm1]].T.values
                x2, y2, z2 = df[self.landmarks[lm2]].T.values

                distance = np.sqrt((x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2)
                df[feature_name] = distance
                distances_created.append(feature_name)

        print(f"      ✅ {len(distances_created)} distancias creadas")
        return distances_created

    def _create_angle_features(self, df):
        """Crear características de ángulos articulares"""
        print("   📐 Creando ángulos articulares...")

        angles_created = []

        # Ángulos articulares importantes
        angle_definitions = [
            ('L_shoulder', 'L_elbow', 'L_wrist', 'L_elbow_angle'),
            ('R_shoulder', 'R_elbow', 'R_wrist', 'R_elbow_angle'),
            ('L_hip', 'L_knee', 'L_ankle', 'L_knee_angle'),
            ('R_hip', 'R_knee', 'R_ankle', 'R_knee_angle')
        ]

        for p1, p2, p3, feature_name in angle_definitions:
            if all(all(col in df.columns for col in self.landmarks[lm]) for lm in [p1, p2, p3]):

                angle = self._calculate_angle(df, p1, p2, p3)
                if angle is not None:
                    df[feature_name] = angle
                    angles_created.append(feature_name)

        print(f"      ✅ {len(angles_created)} ángulos creados")
        return angles_created

    def _calculate_angle(self, df, point1, point2, point3):
        """Calcular ángulo entre 3 puntos"""
        try:
            # Vectores
            x1, y1 = df[f"{point1}_x"], df[f"{point1}_y"]
            x2, y2 = df[f"{point2}_x"], df[f"{point2}_y"]
            x3, y3 = df[f"{point3}_x"], df[f"{point3}_y"]

            # Vector 1: point2 -> point1
            v1_x, v1_y = x1 - x2, y1 - y2
            # Vector 2: point2 -> point3
            v2_x, v2_y = x3 - x2, y3 - y2

            # Calcular ángulo usando producto punto
            dot_product = v1_x * v2_x + v1_y * v2_y
            magnitude1 = np.sqrt(v1_x**2 + v1_y**2)
            magnitude2 = np.sqrt(v2_x**2 + v2_y**2)

            # Evitar división por cero
            magnitude_product = magnitude1 * magnitude2
            valid_mask = magnitude_product > 1e-8

            angles = np.zeros(len(df))
            angles[valid_mask] = np.arccos(
                np.clip(dot_product[valid_mask] / magnitude_product[valid_mask], -1, 1)
            )

            return np.degrees(angles)  # Convertir a grados

        except Exception as e:
            print(f"❌ Error calculando ángulo: {e}")
            return None

    def _create_ratio_features(self, df):
        """Crear ratios y proporciones corporales"""
        print("   📊 Creando ratios corporales...")

        ratios_created = []

        # Ratios corporales útiles
        if all(col in df.columns for col in ['shoulder_width', 'hip_width']):
            df['shoulder_hip_ratio'] = df['shoulder_width'] / (df['hip_width'] + 1e-8)
            ratios_created.append('shoulder_hip_ratio')

        if all(col in df.columns for col in ['L_torso_length', 'L_thigh_length']):
            df['torso_thigh_ratio'] = df['L_torso_length'] / (df['L_thigh_length'] + 1e-8)
            ratios_created.append('torso_thigh_ratio')

        # Altura aproximada (hombro a tobillo)
        if all(col in df.columns for col in ['L_shoulder_y', 'L_ankle_y']):
            df['body_height_approx'] = abs(df['L_ankle_y'] - df['L_shoulder_y'])
            ratios_created.append('body_height_approx')

        print(f"      ✅ {len(ratios_created)} ratios creados")
        return ratios_created

    def _create_center_mass_features(self, df):
        """Crear características de centro de masa"""
        print("   ⚖️ Creando centro de masa...")

        center_features = []

        # Centro de masa corporal (promedio caderas)
        if all(col in df.columns for col in ['L_hip_x', 'R_hip_x', 'L_hip_y', 'R_hip_y']):
            df['center_mass_x'] = (df['L_hip_x'] + df['R_hip_x']) / 2
            df['center_mass_y'] = (df['L_hip_y'] + df['R_hip_y']) / 2
            center_features.extend(['center_mass_x', 'center_mass_y'])

        # Centro torso superior (promedio hombros)
        if all(col in df.columns for col in ['L_shoulder_x', 'R_shoulder_x', 'L_shoulder_y', 'R_shoulder_y']):
            df['upper_center_x'] = (df['L_shoulder_x'] + df['R_shoulder_x']) / 2
            df['upper_center_y'] = (df['L_shoulder_y'] + df['R_shoulder_y']) / 2
            center_features.extend(['upper_center_x', 'upper_center_y'])

        print(f"      ✅ {len(center_features)} centros de masa creados")
        return center_features


# ============================================
# APLICAR FEATURE ENGINEERING A CADA DATASET
# ============================================

# Crear feature engineer
geo_engineer = GeometricFeatureEngineer()

# CRÍTICO: Aplicar a cada dataset POR SEPARADO (sin mezclar)
if all(df is not None for df in [train_df, val_df, test_df]):

    # Train
    train_geo, train_features = geo_engineer.create_geometric_features(train_df, 'TRAIN')

    # Validation
    val_geo, val_features = geo_engineer.create_geometric_features(val_df, 'VALIDATION')

    # Test
    test_geo, test_features = geo_engineer.create_geometric_features(test_df, 'TEST')

    print(f"\n" + "="*60)
    print(f"✅ FEATURE ENGINEERING COMPLETADO PARA TODOS LOS DATASETS")
    print(f"="*60)

    print(f"\n📊 RESUMEN:")
    print(f"   Train: {train_geo.shape[0]:,} × {train_geo.shape[1]} (features)")
    print(f"   Val:   {val_geo.shape[0]:,} × {val_geo.shape[1]} (features)")
    print(f"   Test:  {test_geo.shape[0]:,} × {test_geo.shape[1]} (features)")

    print(f"\n🔧 Features geométricas añadidas: {len(train_features)}")
    print(f"   {', '.join(train_features[:5])}...")

    print(f"\n🚀 Listo para Preprocessing Pipeline")

else:
    print("❌ Error: Datasets no cargados correctamente")



🔶 FEATURE ENGINEERING: TRAIN
   📏 Creando distancias entre landmarks...
      ✅ 8 distancias creadas
   📐 Creando ángulos articulares...
      ✅ 4 ángulos creados
   📊 Creando ratios corporales...
      ✅ 3 ratios creados
   ⚖️ Creando centro de masa...
      ✅ 4 centros de masa creados

✅ FEATURES CREADAS PARA TRAIN:
   📊 Total nuevas features: 19
   📐 Dimensiones finales: 5,418 × 86

🔶 FEATURE ENGINEERING: VALIDATION
   📏 Creando distancias entre landmarks...
      ✅ 8 distancias creadas
   📐 Creando ángulos articulares...
      ✅ 4 ángulos creados
   📊 Creando ratios corporales...
      ✅ 3 ratios creados
   ⚖️ Creando centro de masa...
      ✅ 4 centros de masa creados

✅ FEATURES CREADAS PARA VALIDATION:
   📊 Total nuevas features: 19
   📐 Dimensiones finales: 967 × 85

🔶 FEATURE ENGINEERING: TEST
   📏 Creando distancias entre landmarks...
      ✅ 8 distancias creadas
   📐 Creando ángulos articulares...
      ✅ 4 ángulos creados
   📊 Creando ratios corporales...
      ✅ 3 ratios 

## Paso 4: Pipeline de Preprocessing
Crear pipeline automatizado para normalización, encoding y preparación final.


In [5]:
# ============================================
# PASO 4: PREPROCESSING PIPELINE (SIN LEAKAGE)
# ============================================

class DataPreprocessingPipeline:
    """Pipeline completo: FIT en train, TRANSFORM en val/test"""

    def __init__(self):
        self.scaler = StandardScaler()
        self.label_encoder = LabelEncoder()
        self.pca = None
        self.feature_cols = None
        self.preprocessing_stats = {}

    def fit_transform_train(self, train_df, use_pca=True, n_components=0.95):
        """
        AJUSTAR pipeline en train y transformar
        """
        print("\n🔧 PREPROCESSING: AJUSTE EN TRAIN")
        print("=" * 60)

        print(f"📊 TRAIN INPUT:")
        print(f"   Frames: {len(train_df):,}")
        print(f"   Columnas: {len(train_df.columns)}")

        # 1. Separar features y labels
        X_train, y_train = self._separate_features_labels(train_df, is_train=True)

        if X_train is None:
            print("❌ Error separando features")
            return None, None

        print(f"\n📊 DATOS EXTRAÍDOS:")
        print(f"   Features (X): {X_train.shape}")
        print(f"   Labels (y): {len(y_train):,}")
        print(f"   Feature columns: {len(self.feature_cols)}")

        # 2. Label encoding
        y_encoded = self.label_encoder.fit_transform(y_train)

        print(f"\n🏷️  LABEL ENCODING (FIT EN TRAIN):")
        for i, class_name in enumerate(self.label_encoder.classes_):
            count = np.sum(y_encoded == i)
            pct = count / len(y_encoded) * 100
            print(f"   {i}: {class_name:20s} → {count:,} ({pct:.1f}%)")

        # 3. Limpiar features
        print(f"\n🧹 LIMPIEZA:")
        nan_count = np.isnan(X_train).sum()
        if nan_count > 0:
            print(f"   ⚠️  NaN encontrados: {nan_count}")
            X_train = np.nan_to_num(X_train, nan=0.0, posinf=1.0, neginf=-1.0)
            print(f"   ✅ NaN reemplazados")
        else:
            print(f"   ✅ Sin NaN")

        # 4. Normalización (FIT en train)
        X_scaled = self.scaler.fit_transform(X_train)
        print(f"   ✅ StandardScaler FIT en train")
        print(f"      Mean: {self.scaler.mean_[:3]} ... (primeras 3)")
        print(f"      Std:  {self.scaler.scale_[:3]} ... (primeras 3)")

        # 5. PCA (FIT en train)
        if use_pca:
            self.pca = PCA(n_components=n_components, random_state=42)
            X_final = self.pca.fit_transform(X_scaled)

            variance = self.pca.explained_variance_ratio_.sum()
            print(f"   ✅ PCA FIT en train:")
            print(f"      {X_train.shape[1]} → {X_final.shape[1]} features")
            print(f"      Varianza: {variance*100:.1f}%")
        else:
            X_final = X_scaled

        # Estadísticas
        self.preprocessing_stats['train'] = {
            'samples': X_final.shape[0],
            'original_features': X_train.shape[1],
            'final_features': X_final.shape[1],
            'classes': len(self.label_encoder.classes_)
        }

        print(f"\n✅ TRAIN PROCESADO: {X_final.shape}")

        return X_final, y_encoded

    def transform_test(self, test_df, dataset_name='Test'):
        """
        TRANSFORMAR val/test usando pipeline YA AJUSTADO en train
        """
        print(f"\n🔄 PREPROCESSING: TRANSFORM EN {dataset_name.upper()}")
        print("=" * 60)

        print(f"📊 {dataset_name.upper()} INPUT:")
        print(f"   Frames: {len(test_df):,}")

        # 1. Separar features y labels (usando MISMAS columnas que train)
        X_test, y_test = self._separate_features_labels(test_df, is_train=False)

        if X_test is None:
            print(f"❌ Error separando features de {dataset_name}")
            return None, None

        print(f"   Features (X): {X_test.shape}")
        print(f"   Labels (y): {len(y_test):,}")

        # 2. Label encoding (TRANSFORM, no fit)
        y_encoded = self.label_encoder.transform(y_test)

        # 3. Limpiar
        X_test = np.nan_to_num(X_test, nan=0.0, posinf=1.0, neginf=-1.0)

        # 4. Normalización (TRANSFORM con scaler de train)
        X_scaled = self.scaler.transform(X_test)  # ← NO fit_transform
        print(f"   ✅ StandardScaler TRANSFORM (usando stats de train)")

        # 5. PCA (TRANSFORM con PCA de train)
        if self.pca:
            X_final = self.pca.transform(X_scaled)  # ← NO fit_transform
            print(f"   ✅ PCA TRANSFORM: {X_test.shape[1]} → {X_final.shape[1]}")
        else:
            X_final = X_scaled

        print(f"\n✅ {dataset_name.upper()} PROCESADO: {X_final.shape}")

        return X_final, y_encoded

    def _separate_features_labels(self, df, is_train=True):
        """Separar features y labels"""

        # Columnas metadata (NO son features)
        metadata_cols = ['activity', 'data_type', 'split', 'frame_number',
                        'video_file', 'source', 'augmentation_type']

        if is_train:
            # En train, identificar feature_cols por primera vez
            self.feature_cols = [col for col in df.columns
                                if col not in metadata_cols]

            print(f"   🔍 Identificadas {len(self.feature_cols)} feature columns")
        else:
            # En val/test, usar MISMAS columnas que train
            if self.feature_cols is None:
                print("   ❌ ERROR: Debes procesar train primero")
                return None, None

        # Verificar que todas las columnas existen
        missing_cols = [col for col in self.feature_cols if col not in df.columns]
        if missing_cols:
            print(f"   ⚠️  Columnas faltantes: {len(missing_cols)}")
            # Usar solo columnas disponibles
            available_cols = [col for col in self.feature_cols if col in df.columns]
            X = df[available_cols].values
        else:
            X = df[self.feature_cols].values

        # Labels
        if 'activity' not in df.columns:
            print("   ❌ No hay columna 'activity'")
            return None, None

        y = df['activity'].values

        return X, y


# ============================================
# EJECUTAR PREPROCESSING CON SEPARACIÓN CORRECTA
# ============================================

# Crear pipeline
preprocessor = DataPreprocessingPipeline()

# PASO CRÍTICO: FIT en train, TRANSFORM en val/test
if all(df is not None for df in [train_geo, val_geo, test_geo]):

    # 1. FIT_TRANSFORM en TRAIN
    X_train, y_train = preprocessor.fit_transform_train(
        train_geo,
        use_pca=True,
        n_components=0.95
    )

    # 2. TRANSFORM en VALIDATION (usando pipeline de train)
    X_val, y_val = preprocessor.transform_test(val_geo, 'Validation')

    # 3. TRANSFORM en TEST (usando pipeline de train)
    X_test, y_test = preprocessor.transform_test(test_geo, 'Test')

    # Verificación final
    if all(X is not None for X in [X_train, X_val, X_test]):
        print(f"\n" + "="*60)
        print(f"✅ PREPROCESSING COMPLETADO SIN LEAKAGE")
        print(f"="*60)

        print(f"\n📊 DATASETS FINALES:")
        print(f"   Train: {X_train.shape[0]:,} × {X_train.shape[1]}")
        print(f"   Val:   {X_val.shape[0]:,} × {X_val.shape[1]}")
        print(f"   Test:  {X_test.shape[0]:,} × {X_test.shape[1]}")

        print(f"\n🔒 GARANTÍA SIN LEAKAGE:")
        print(f"   ✓ Scaler FIT solo en train")
        print(f"   ✓ PCA FIT solo en train")
        print(f"   ✓ Val/Test solo TRANSFORMADOS")

        print(f"\n🚀 Listos para guardar y entregar a Tomás")

    else:
        print(f"\n❌ Error en algún dataset")

else:
    print("❌ Datasets geométricos no disponibles")



🔧 PREPROCESSING: AJUSTE EN TRAIN
📊 TRAIN INPUT:
   Frames: 5,418
   Columnas: 86
   🔍 Identificadas 83 feature columns

📊 DATOS EXTRAÍDOS:
   Features (X): (5418, 83)
   Labels (y): 5,418
   Feature columns: 83

🏷️  LABEL ENCODING (FIT EN TRAIN):
   0: caminar_hacia        → 1,290 (23.8%)
   1: caminar_regreso      → 1,032 (19.0%)
   2: girar                → 1,032 (19.0%)
   3: ponerse_pie          → 1,032 (19.0%)
   4: sentarse             → 1,032 (19.0%)

🧹 LIMPIEZA:
   ✅ Sin NaN
   ✅ StandardScaler FIT en train
      Mean: [ 0.48664878  0.36699865 -0.12852494] ... (primeras 3)
      Std:  [0.09141731 0.07074265 0.13676313] ... (primeras 3)
   ✅ PCA FIT en train:
      83 → 16 features
      Varianza: 95.1%

✅ TRAIN PROCESADO: (5418, 16)

🔄 PREPROCESSING: TRANSFORM EN VALIDATION
📊 VALIDATION INPUT:
   Frames: 967
   Features (X): (967, 83)
   Labels (y): 967
   ✅ StandardScaler TRANSFORM (usando stats de train)
   ✅ PCA TRANSFORM: 83 → 16

✅ VALIDATION PROCESADO: (967, 16)

🔄 PREPR

## Paso 5: Guardar

In [7]:
# ============================================
# PASO 5: GUARDAR DATOS FINALES PARA MODELADO
# ============================================

import os
import joblib
import json
from datetime import datetime

def save_ml_ready_data():
    """
    Guardar datasets procesados y pipelines para Tomás
    """
    print("\n💾 GUARDANDO DATOS PARA MODELADO")
    print("=" * 60)

    # Crear directorios
    os.makedirs('data/ml_ready', exist_ok=True)
    os.makedirs('data/models', exist_ok=True)

    # 1. Guardar arrays numpy (.npy)
    print("\n📁 Guardando arrays numpy...")
    np.save('data/ml_ready/X_train.npy', X_train)
    np.save('data/ml_ready/y_train.npy', y_train)
    np.save('data/ml_ready/X_val.npy', X_val)
    np.save('data/ml_ready/y_val.npy', y_val)
    np.save('data/ml_ready/X_test.npy', X_test)
    np.save('data/ml_ready/y_test.npy', y_test)

    print(f"   ✅ X_train.npy: {X_train.shape}")
    print(f"   ✅ y_train.npy: {y_train.shape}")
    print(f"   ✅ X_val.npy:   {X_val.shape}")
    print(f"   ✅ y_val.npy:   {y_val.shape}")
    print(f"   ✅ X_test.npy:  {X_test.shape}")
    print(f"   ✅ y_test.npy:  {y_test.shape}")

    # 2. Guardar pipelines (.pkl)
    print("\n🔧 Guardando pipelines...")
    joblib.dump(preprocessor.scaler, 'data/models/scaler.pkl')
    joblib.dump(preprocessor.label_encoder, 'data/models/label_encoder.pkl')
    joblib.dump(preprocessor.pca, 'data/models/pca.pkl')

    print(f"   ✅ scaler.pkl (StandardScaler)")
    print(f"   ✅ label_encoder.pkl (LabelEncoder)")
    print(f"   ✅ pca.pkl (PCA)")

    # 3. Metadata completa
    print("\n📋 Creando metadata...")

    metadata = {
        'creation_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'project': 'Sistema Anotación Video - Entrega 2',
        'datasets': {
            'train': {
                'samples': int(X_train.shape[0]),
                'features': int(X_train.shape[1]),
                'original_frames': 4509,
                'smote_frames': 909,
                'balance_ratio': 0.800,
                'classes_distribution': {
                    'caminar_hacia': 1290,
                    'caminar_regreso': 1032,
                    'girar': 1032,
                    'ponerse_pie': 1032,
                    'sentarse': 1032
                }
            },
            'validation': {
                'samples': int(X_val.shape[0]),
                'features': int(X_val.shape[1]),
                'data_type': '100% real',
                'balance_ratio': 0.509
            },
            'test': {
                'samples': int(X_test.shape[0]),
                'features': int(X_test.shape[1]),
                'data_type': '100% real',
                'balance_ratio': 0.509
            }
        },
        'preprocessing': {
            'original_features': 83,
            'final_features': int(X_train.shape[1]),
            'pca_variance_explained': float(preprocessor.pca.explained_variance_ratio_.sum()),
            'scaler': 'StandardScaler',
            'pca_components': int(X_train.shape[1])
        },
        'classes': preprocessor.label_encoder.classes_.tolist(),
        'quality_assurance': {
            'no_data_leakage': True,
            'scaler_fit_on': 'train_only',
            'pca_fit_on': 'train_only',
            'synthetic_ratio_train': 0.168,
            'test_completely_real': True
        }
    }

    with open('data/ml_ready/metadata.json', 'w') as f:
        json.dump(metadata, f, indent=2)

    print(f"   ✅ metadata.json")

    # 4. README para Tomás
    print("\n📄 Creando README...")

    readme_content = """
============================================================
DATASETS LISTOS PARA MODELADO - ENTREGA 2
============================================================

Fecha: """ + datetime.now().strftime('%Y-%m-%d %H:%M:%S') + """

ARCHIVOS:
---------
• X_train.npy: """ + f"{X_train.shape[0]:,} × {X_train.shape[1]}" + """ (entrenamiento)
• y_train.npy: """ + f"{y_train.shape[0]:,}" + """ labels
• X_val.npy:   """ + f"{X_val.shape[0]:,} × {X_val.shape[1]}" + """ (validación hiperparámetros)
• y_val.npy:   """ + f"{y_val.shape[0]:,}" + """ labels
• X_test.npy:  """ + f"{X_test.shape[0]:,} × {X_test.shape[1]}" + """ (evaluación final)
• y_test.npy:  """ + f"{y_test.shape[0]:,}" + """ labels

PIPELINES:
----------
• scaler.pkl: StandardScaler (ajustado en train)
• label_encoder.pkl: Codificación de actividades
• pca.pkl: PCA 83→16 features (95.1% varianza)

CARACTERÍSTICAS:
----------------
✓ Sin data leakage (fit solo en train)
✓ Balance train: 0.800 (SMOTE aplicado)
✓ Val/Test: 100% datos reales
✓ PCA: 83 features → 16 componentes
✓ Varianza explicada: 95.1%

CLASES:
-------
0: caminar_hacia
1: caminar_regreso
2: girar
3: ponerse_pie
4: sentarse

USO EN MODELADO:
----------------
```
import numpy as np
import joblib

# Cargar datos
X_train = np.load('data/ml_ready/X_train.npy')
y_train = np.load('data/ml_ready/y_train.npy')
X_val = np.load('data/ml_ready/X_val.npy')
y_val = np.load('data/ml_ready/y_val.npy')
X_test = np.load('data/ml_ready/X_test.npy')
y_test = np.load('data/ml_ready/y_test.npy')

# Cargar pipelines (si necesitas transformar nuevos datos)
scaler = joblib.load('data/models/scaler.pkl')
label_encoder = joblib.load('data/models/label_encoder.pkl')
pca = joblib.load('data/models/pca.pkl')

# Entrenar modelo
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluar
from sklearn.metrics import classification_report
val_pred = model.predict(X_val)
print(classification_report(y_val, val_pred,
                           target_names=label_encoder.classes_))
```

WORKFLOW RECOMENDADO:
---------------------
1. Entrenar modelos con X_train, y_train
2. Optimizar hiperparámetros con X_val, y_val
3. Evaluación FINAL con X_test, y_test (SOLO UNA VEZ)

GARANTÍAS:
----------
✓ Test set nunca visto durante preprocessing
✓ Scaler/PCA ajustados SOLO en train
✓ Balance train: 0.800
✓ Datos reales en val/test para evaluación genuina

============================================================
"""

    with open('data/ml_ready/README.txt', 'w') as f:
        f.write(readme_content)

    print(f"   ✅ README.txt")

    # Tamaños de archivos
    print("\n💾 TAMAÑOS DE ARCHIVOS:")
    for filename in ['X_train.npy', 'y_train.npy', 'X_val.npy', 'y_val.npy', 'X_test.npy', 'y_test.npy']:
        path = f'data/ml_ready/{filename}'
        size_mb = os.path.getsize(path) / (1024**2)
        print(f"   {filename:15s}: {size_mb:6.2f} MB")

    print("\n" + "🎉" * 30)
    print("🎉 NOTEBOOK 4 COMPLETADO - DATOS LISTOS PARA TOMÁS 🎉")
    print("🎉" * 30)

    print("\n📂 UBICACIÓN: data/ml_ready/")
    print("\n✅ Tomás puede empezar a entrenar modelos ahora")

# Ejecutar guardado
save_ml_ready_data()


💾 GUARDANDO DATOS PARA MODELADO

📁 Guardando arrays numpy...
   ✅ X_train.npy: (5418, 16)
   ✅ y_train.npy: (5418,)
   ✅ X_val.npy:   (967, 16)
   ✅ y_val.npy:   (967,)
   ✅ X_test.npy:  (967, 16)
   ✅ y_test.npy:  (967,)

🔧 Guardando pipelines...
   ✅ scaler.pkl (StandardScaler)
   ✅ label_encoder.pkl (LabelEncoder)
   ✅ pca.pkl (PCA)

📋 Creando metadata...
   ✅ metadata.json

📄 Creando README...
   ✅ README.txt

💾 TAMAÑOS DE ARCHIVOS:
   X_train.npy    :   0.66 MB
   y_train.npy    :   0.04 MB
   X_val.npy      :   0.12 MB
   y_val.npy      :   0.01 MB
   X_test.npy     :   0.12 MB
   y_test.npy     :   0.01 MB

🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉
🎉 NOTEBOOK 4 COMPLETADO - DATOS LISTOS PARA TOMÁS 🎉
🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉

📂 UBICACIÓN: data/ml_ready/

✅ Tomás puede empezar a entrenar modelos ahora
